# Le modèle du Brusselator

Le modèle du brusselator est un exemple de modèle de reaction chimique oscillante, il s'écrit sous la forme :

$$
\left\{\begin{aligned}
d_t y_1 & = 1 - (b+1) y_1 + a y_1^2y_2\\
d_t y_2 & = b y_1 - a y_1^2y_2
\end{aligned}\right.
$$

In [ ]:
import numpy as np

from scipy.integrate import solve_ivp

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.models import PrintfTickFormatter

from ipywidgets import interact, IntSlider, Dropdown

from mylib.model import brusselator_model
import mylib.integration as integration

output_notebook(hide_banner=True)

## Solution quasi-exacte

La solution quasi-exacte est obtenue en utilisant une méthode de Runge-Kutta d'ordre 5 avec pas de temps adapatitifs.

In [ ]:
def plot_quasi_exact_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5, 3)
    
    sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12)

    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    fig_sol.line(sol.t, sol.y[0], legend="y1", line_width=2)    
    fig_sol.line(sol.t, sol.y[1], legend="y2", line_width=2, color="Green")
    fig_sol.legend.location = "top_left"
    
    show(fig_sol)
    
plot_quasi_exact_sol()

## Caractérisation de la raideur

In [ ]:
def plot_eigenvalue():

    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    jac = bm.jac
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5 , 3)
    
    sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12)
    
    eig_vals = np.zeros((sol.t.size, 2), dtype=np.complex_)
    for it, yi in enumerate(sol.y.transpose()):
        eig_vals[it] = np.linalg.eigvals(jac(0, yi))

    lambda1 = eig_vals[:, 0]
    lambda2 = eig_vals[:, 1]
    
    fig_real = figure(x_range=(tini, tend), plot_height=300, plot_width=900, 
                      title = "Real part of eigenvalues (click on legend entries to hide corresponding plot)")
    fig_imag = figure(x_range=(tini, tend), plot_height=300, plot_width=900, 
                      title = "Imaginary part of eigenvalues (click on lengend to hide corresponding plot)")
            
    fig_real.line(sol.t, np.real(lambda1), line_width=2, legend="lambda1")
    fig_real.line(sol.t, np.real(lambda2), line_width=2, color="Green", legend="lambda2")
    fig_real.legend.click_policy="hide"

    fig_imag.line(sol.t, np.imag(lambda1), line_width=2, legend="lambda1")
    fig_imag.line(sol.t, np.imag(lambda2), line_width=2, color="Green", legend="lambda2")
    fig_imag.legend.click_policy="hide"

    show(column(fig_real, fig_imag))
    
plot_eigenvalue()

## Méthode de Runge-Kutta

In [ ]:
def plot_rk_methods():

    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    
    tini = 0.
    tend = 20.
    
    yini = (1.5 , 3)
    
    nt = 1000
    sol_rk = integration.forward_euler(tini, tend, nt, yini, fcn)
    
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12, t_eval=sol_rk.t)
    y1_err = np.abs(sol_exa.y[0] - sol_rk.y[0])
    y2_err = np.abs(sol_exa.y[1] - sol_rk.y[1])
    
    # plot exact and rk solutions 
    fig_sol = figure(x_range=(tini, tend), plot_height=300, plot_width=900, title="Solution")
    plt_sol_y1 = fig_sol.x(sol_rk.t, sol_rk.y[0], line_width=2, legend="y1")
    plt_exa_y1 = fig_sol.line(sol_rk.t, sol_exa.y[0], color="Grey")
    plt_sol_y2 = fig_sol.x(sol_rk.t, sol_rk.y[0], line_width=2, color="Green", legend="y2")
    plt_exa_y1 = fig_sol.line(sol_rk.t, sol_exa.y[1], color="Grey")
    fig_sol.legend.location = "top_left"

    # plot global error
    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=900, title="Global error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")
    plt_err_y1 = fig_err.x(sol_rk.t, y1_err, line_width=2, legend="y1")
    plt_err_y2 = fig_err.x(sol_rk.t, y2_err, line_width=2, color="Green", legend="y2")
    fig_err.legend.location = "top_left"

    show(column(fig_sol, fig_err), notebook_handle=True)
    
    def update(method, nt):
        t = np.linspace(tini, tend, nt)
        if method == "rk1":
            sol_rk = integration.forward_euler(tini, tend, nt, yini, fcn)
        elif method == "rk2":
            sol_rk = integration.rk2(tini, tend, nt, yini, fcn)
        elif method == "rk3":
            sol_rk = integration.rk3(tini, tend, nt, yini, fcn)
        elif method == "rk4":
            sol_rk = integration.rk4(tini, tend, nt, yini, fcn)        

        sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12, t_eval=sol_rk.t)
        y1_err = np.abs(sol_exa.y[0] - sol_rk.y[0])
        y2_err = np.abs(sol_exa.y[1] - sol_rk.y[1])
        
        plt_sol_y1.data_source.data = dict(x=sol_rk.t, y=sol_rk.y[0])
        plt_sol_y2.data_source.data = dict(x=sol_rk.t, y=sol_rk.y[1])
        plt_err_y1.data_source.data = dict(x=sol_rk.t, y=y1_err)
        plt_err_y2.data_source.data = dict(x=sol_rk.t, y=y2_err)
        push_notebook()
        
    interact(update,
             nt=IntSlider(min=100, max=5000, value=nt, step=100, continuous_update=False),
             method=Dropdown(options=["rk1", "rk2", "rk3", "rk4"], value='rk1', description='Method:'))

plot_rk_methods()

## Méthode d'Euler explicite

In [ ]:
def plot_backward_euler_method():

    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    
    tini = 0.
    tend = 20.
    
    yini = (1.5 , 3)
    
    nt = 1000
    sol_be = integration.backward_euler(tini, tend, nt, yini, fcn)
    
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12, t_eval=sol_be.t)
    y1_err = np.abs(sol_exa.y[0] - sol_be.y[0])
    y2_err = np.abs(sol_exa.y[1] - sol_be.y[1])
    
    # plot exact and backward euler solution 
    fig_sol = figure(x_range=(tini, tend), plot_height=300, plot_width=900, title="Solution")
    plt_sol_y1 = fig_sol.x(sol_be.t, sol_be.y[0], line_width=2, legend="y1")
    plt_exa_y1 = fig_sol.line(sol_be.t, sol_exa.y[0], color="Grey")
    plt_sol_y2 = fig_sol.x(sol_be.t, sol_be.y[0], line_width=2, color="Green", legend="y2")
    plt_exa_y1 = fig_sol.line(sol_be.t, sol_exa.y[1], color="Grey")
    fig_sol.legend.location = "top_left"

    # plot global error
    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=900, title="Global error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")
    plt_err_y1 = fig_err.x(sol_be.t, y1_err, line_width=2, legend="y1")
    plt_err_y2 = fig_err.x(sol_be.t, y2_err, line_width=2, color="Green", legend="y2")
    fig_err.legend.location = "top_left"

    show(column(fig_sol, fig_err), notebook_handle=True)
    
    def update(nt):
        sol_be = integration.backward_euler(tini, tend, nt, yini, fcn)
        sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12, t_eval=sol_be.t)
        y1_err = np.abs(sol_exa.y[0] - sol_be.y[0])
        y2_err = np.abs(sol_exa.y[1] - sol_be.y[1])
        
        plt_sol_y1.data_source.data = dict(x=sol_be.t, y=sol_be.y[0])
        plt_sol_y2.data_source.data = dict(x=sol_be.t, y=sol_be.y[1])
        plt_err_y1.data_source.data = dict(x=sol_be.t, y=y1_err)
        plt_err_y2.data_source.data = dict(x=sol_be.t, y=y2_err)
        push_notebook()
        
    interact(update, nt=IntSlider(min=100, max=5000, value=nt, step=100, continuous_update=False))

plot_backward_euler_method()

## Méthode emboîtée

 ### Methode basée sur méthode de la "règle 3/8" (ordre 4)

In [ ]:
def plot_rk_embedded_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    
    tini = 0. 
    tend = 20.
    nt = 200
    
    yini = (1.5 , 3)
    
    sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12)

    tol = 1.e-4
    sol_rk_emb = integration.rk_embedded(tini, tend, yini, fcn, tol)
    
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12, t_eval=sol_rk_emb.t)
    glob_err = np.empty(sol_rk_emb.t.size)
    for i in range(sol_rk_emb.t.size):
        glob_err[i] = np.sqrt(1./2) * np.linalg.norm(sol_exa.y[:,i] - sol_rk_emb.y[:,i])

    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    fig_sol.line(sol.t, sol.y[0], color="Grey")
    fig_sol.line(sol.t, sol.y[1], color="Grey")
    plt_y1 = fig_sol.x(sol_rk_emb.t, sol_rk_emb.y[0], legend="y1", line_width=2)    
    plt_y2 = fig_sol.x(sol_rk_emb.t, sol_rk_emb.y[1], legend="y2", line_width=2, color="Green")
    fig_sol.legend.location = "top_left"
  
    fig_dt = figure(x_range=(tini, tend), plot_height=300, plot_width=950, y_axis_type="log", 
                    title = "Accepted (black) and rejected (red) times step") 
    plt_dt = fig_dt.square(sol_rk_emb.t[0:-1], sol_rk_emb.dt, color="Black")
    plt_line_dt = fig_dt.line(sol_rk_emb.t[0:-1], sol_rk_emb.dt, color="Black")
    plt_dt_rej = fig_dt.circle_x(sol_rk_emb.t_rej, sol_rk_emb.dt_rej, color="Crimson")

    fig_err = figure(x_range=(tini, tend), y_axis_type="log", width=950, height=300, title="Error")
    plt_loc_err = fig_err.x(sol_rk_emb.t, sol_rk_emb.loc_err_est, legend="estimate local error", line_width=2)
    plt_loc_err_exa = fig_err.circle(sol_rk_emb.t, sol_rk_emb.loc_err, legend="local error", fill_alpha=0., color="crimson")
    plt_glob_err = fig_err.square(sol_rk_emb.t, glob_err, legend="global error", fill_alpha=0, color="green")
    plt_tol = fig_err.line(sol_rk_emb.t, tol*np.ones(sol_rk_emb.t.size), color="grey", line_dash="dotted")    
    fig_err.legend.location = "bottom_left"
    fig_err.legend.orientation = "horizontal"

    show(column(fig_sol, fig_dt, fig_err), notebook_handle=True)
    
    def update(tol):
        
        sol_rk_emb = integration.rk_embedded(tini, tend, yini, fcn, tol)
        
        sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12, t_eval=sol_rk_emb.t)
        glob_err = np.empty(sol_rk_emb.t.size)
        for i in range(sol_rk_emb.t.size):
            glob_err[i] = np.sqrt(1./2) * np.linalg.norm(sol_exa.y[:,i] - sol_rk_emb.y[:,i])


        plt_y1.data_source.data = dict(x=sol_rk_emb.t, y=sol_rk_emb.y[0])
        plt_y2.data_source.data = dict(x=sol_rk_emb.t, y=sol_rk_emb.y[1]) 

        plt_dt.data_source.data = dict(x=sol_rk_emb.t[0:-1], y=sol_rk_emb.dt)
        plt_line_dt.data_source.data = dict(x=sol_rk_emb.t[0:-1], y=sol_rk_emb.dt) 
        plt_dt_rej.data_source.data = dict(x=sol_rk_emb.t_rej, y=sol_rk_emb.dt_rej)
        
        plt_loc_err.data_source.data = dict(x=sol_rk_emb.t, y=sol_rk_emb.loc_err_est)
        plt_loc_err_exa.data_source.data  = dict(x=sol_rk_emb.t, y=sol_rk_emb.loc_err)
        plt_glob_err.data_source.data = dict(x=sol_rk_emb.t, y=glob_err)
        plt_tol.data_source.data = dict(x=sol_rk_emb.t, y=tol*np.ones(sol_rk_emb.t.size))
        
        print("   Number of time step : " + str(sol_rk_emb.t.size-1))
        print("   Number of function evaluations : " + str(sol_rk_emb.nfev))

        push_notebook()
        
    dtol={'1.e-1':1.e-1, '1.e-2':1.e-2, '1.e-3':1.e-3, '1.e-4':1.e-4, '1.e-5':1.e-5, '1.e-6':1.e-6}     
    interact(update, tol=Dropdown(options=dtol, value=1.e-4, description='tol'))


plot_rk_embedded_sol()

## Radau5

In [ ]:
def plot_radau5_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5, 3)
    
    sol_rad = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=1.e-4, atol=1.e-4)
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12, t_eval=sol_rad.t)
    
    y1_err = np.abs(sol_exa.y[0] - sol_rad.y[0])
    y2_err = np.abs(sol_exa.y[1] - sol_rad.y[1])

    # plot radau5 solution
    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    fig_sol.line(sol_exa.t, sol_exa.y[0], color="Grey")
    fig_sol.line(sol_exa.t, sol_exa.y[1], color="Grey")
    plt_sol_y1 = fig_sol.x(sol_rad.t, sol_rad.y[0], legend="y1", line_width=2)    
    plt_sol_y2 = fig_sol.x(sol_rad.t, sol_rad.y[1], legend="y2", line_width=2, color="Green")
    fig_sol.legend.location = "top_left"
    
    # plot global error
    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=950, title="Global error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")
    plt_err_y1 = fig_err.x(sol_rad.t, y1_err, line_width=2, legend="y1")
    plt_err_y2 = fig_err.x(sol_rad.t, y2_err, line_width=2, color="Green", legend="y2")
    fig_err.legend.location = "top_left"

    show(column(fig_sol, fig_err), notebook_handle=True)
    
    def update(tol):
        sol_rad = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=tol, atol=tol)
        sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12, t_eval=sol_rad.t)
        y1_err = np.abs(sol_exa.y[0] - sol_rad.y[0])
        y2_err = np.abs(sol_exa.y[1] - sol_rad.y[1])
        
        plt_sol_y1.data_source.data = dict(x=sol_rad.t, y=sol_rad.y[0])
        plt_sol_y2.data_source.data = dict(x=sol_rad.t, y=sol_rad.y[1])
        plt_err_y1.data_source.data = dict(x=sol_rad.t, y=y1_err)
        plt_err_y2.data_source.data = dict(x=sol_rad.t, y=y2_err)
        
        print("   Number of time step : " + str(sol_rad.t.size-1))
        print("   Number of function evaluations : " + str(sol_rad.nfev))
        print("   Number of Jacobian evaluations : " + str(sol_rad.njev))
        print("   Number of LU decompositions    : " + str(sol_rad.nlu))
        
        push_notebook()
                
    dtol={'1.e-2':1.e-2, '1.e-3':1.e-3, '1.e-4':1.e-4, '1.e-5':1.e-5, '1.e-6':1.e-6, '1.e-7':1.e-7, '1.e-8':1.e-8}     
    interact(update, tol=Dropdown(options=dtol, value=1.e-4, description='tol'))

plot_radau5_sol()